#### Import

In [ ]:
#!/usr/bin/env python3
import time
import numpy as np
from math import *

In [ ]:
import urx
import math3d as m3d
ur = urx.Robot("192.168.1.10")
ur.set_tcp((0, 0, 0, 0, 0, 0))
tcp_acc=0.05
tcp_vel=0.05
time.sleep(0.2)
pose = ur.get_pose()
print(pose)

In [1]:
from ddh import DDGripper

In [2]:
gripper = DDGripper()

reading gripper parameters...
connecting to odrive...
found left finger
found right fingers


#### testing

In [11]:
gripper.disarm()

In [10]:
gripper.arm(200)

In [8]:
gripper.arm(150)
gripper.set_parallel_jaw(10, 0)
time.sleep(0.5)
a2_init = (gripper.left_a2 + gripper.right_a2)/2 

NameError: name 'time' is not defined

In [ ]:
gripper.set_stiffness(50)

In [ ]:
gripper.set_left_a1_phi(-5, 20)
gripper.set_right_a1_phi(5, -20)

In [ ]:
gripper.set_left_a1_phi(-0, 20)
gripper.set_right_a1_phi(0, -20)

In [ ]:
print(gripper.left_a2,gripper.right_a2)

In [ ]:
# check finger collision
while 1:
    a2_cur = (gripper.left_a2 + gripper.right_a2)/2
#     print(a2_cur)
    if a2_cur - a2_init > 0.5:
        print ("Collision detected!")
        break

In [ ]:
print(ur.getl())

In [ ]:
init = (-0.73, 0.17, 0.4, pi, 0, 0)
ur.movel(init, tcp_acc, tcp_vel)

In [ ]:
smack_vel = 100
smack_acc = 0.3
stop_acc = 0.5 #0.5
ur.speedl([0,0,-smack_vel,0,0,0],smack_acc,5)

In [ ]:
ur.speedl([0,0,smack_vel,0,0,0],stop_acc,5)
time.sleep(0.5)
ur.stopl(stop_acc)